In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('booking_data.csv')

# מיון הנתונים
sorted_df = data.sort_values(by=['Snapshot Date', 'TTT', 'LOS'])

# הוספת שדה חדש 'new_order' לסידור הנתונים
current_los = None
page = 0
pages_list = []
new_order = 0
new_order_list = []

for index, row in sorted_df.iterrows():
    if row['LOS'] != current_los:
        page += 1 
        new_order = 1
        current_los = row['LOS']
    else:
        new_order += 1
    new_order_list.append(new_order)
    pages_list.append(page)

sorted_df['new_order'] = new_order_list
sorted_df['page_num'] = pages_list

data = sorted_df.drop(['order'], axis=1)

# שמירת הנתונים לקובץ CSV
data.to_csv('booking_data_sorted.csv', index=False)

# הורדת עמודות מיותרות
data = data.drop(['name', 'checkin_date' ,'checkout_date', 'Snapshot Date'], axis=1)

In [2]:
# חלוקת הנתונים לפי מספרי עמודים זוגיים ואי זוגיים
train_data = data[data['TTT'] % 2 == 0]
test_data = data[data['TTT'] % 2 != 0]

# בחירת עמודות מספריות בלבד, למעט new_order ו-page_num
numeric_columns = train_data.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns.remove('new_order')
numeric_columns.remove('page_num')


In [3]:
from sklearn.preprocessing import StandardScaler

def create_pairs(data, numeric_columns, num_pairs=100):
    pairs = []  # רשימה לאחסון הזוגות
    pages = data['page_num'].unique()  # רשימה של כל העמודים הייחודיים

    scaler = StandardScaler()
    data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
    
    # עבור כל עמוד
    for page in pages:
        page_data = data[data['page_num'] == page]  # נתונים של העמוד הנוכחי
        indices = page_data.index.values  # אינדקסים של השורות בעמוד הנוכחי

        # בדיקה אם יש מספיק מלונות ליצירת זוגות
        if len(indices) < 2:
            continue

        # יצירת מספר קבוע של זוגות אקראיים
        for _ in range(num_pairs):
            idx_a, idx_b = np.random.choice(indices, size=2, replace=False)  # בחירת שני אינדקסים אקראיים
            hotel_a_features = page_data.loc[idx_a, numeric_columns].values  # מאפייני מלון א
            hotel_b_features = page_data.loc[idx_b, numeric_columns].values  # מאפייני מלון ב

            # הקלסיפיקציה היא 1 אם מלון א מדורג גבוה יותר, -1 אם מלון ב מדורג גבוה יותר
            classification = 1 if page_data.loc[idx_a]['new_order'] < page_data.loc[idx_b]['new_order'] else -1

            # הוספת הזוג לרשימה
            pairs.append({
                'hotel_a_features': hotel_a_features,
                'hotel_b_features': hotel_b_features,
                'classification': classification,
                'idx_a': idx_a,
                'idx_b': idx_b
            })

    # החזרת הנתונים כ-DataFrame
    return pd.DataFrame(pairs)

# יצירת זוגות עבור קבוצת האימון
train_pairs = create_pairs(train_data, numeric_columns)


C:\Users\ygastfraind\AppData\Local\Temp\ipykernel_10384\1206910376.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_columns] = scaler.fit_transform(data[numeric_columns])


In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

def prepare_weighted_training_data(pairs, important_features, feature_importance):
    X = []  # רשימה לאחסון הפרשי המאפיינים בין המלונות
    y = []  # רשימה לאחסון הקלסיפיקציה (1 או -1)
    
    for _, row in pairs.iterrows():
        hotel_a_features = pd.Series(row['hotel_a_features'], index=numeric_columns).loc[important_features].values
        hotel_b_features = pd.Series(row['hotel_b_features'], index=numeric_columns).loc[important_features].values
        
        # חישוב הפרשי המאפיינים בין מלון א למלון ב ושקלול לפי חשיבות הפיצ'רים
        diff_features = (hotel_a_features - hotel_b_features) * feature_importance
        X.append(diff_features)
        y.append(row['classification'])
    
    return np.array(X), np.array(y)

# אימון מודל עץ החלטה לחישוב חשיבות הפיצ'רים
tree_model = DecisionTreeClassifier(criterion='gini',max_depth=5)
X_train_base, y_train_base = prepare_weighted_training_data(train_pairs, numeric_columns, np.ones(len(numeric_columns)))
tree_model.fit(X_train_base, y_train_base)
feature_importance = tree_model.feature_importances_

# הדפסת חשיבות הפיצ'רים עם שמות הפיצ'רים
feature_importance_dict = {feature: importance for feature, importance in zip(numeric_columns, feature_importance)}
sorted_features = sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)
for feature, importance in sorted_features:
    print(f"{feature}: {importance}")

# בחירת 5 הפיצ'רים הכי חשובים
top_5_features = [feature for feature, importance in sorted_features[:5]]

# הכנת נתוני האימון עם שקלול לפי 5 הפיצ'רים הכי חשובים
top_5_feature_importance = [feature_importance_dict[feature] for feature in top_5_features]
X_train_weighted, y_train = prepare_weighted_training_data(train_pairs, top_5_features, top_5_feature_importance)
X_train_weighted


price: 0.48689834182362207
num_of_reviews: 0.26923669886810936
grade: 0.1545015944299024
no_prepayment: 0.03875237327540791
distance_from_center: 0.024284331771152055
free_cancellation: 0.023403572568249785
breakfast: 0.0016002713520194686
room_category: 0.0013228159115370123
descriptive_grades: 0.0
stars: 0.0
TTT: 0.0
LOS: 0.0
bed_category: 0.0


array([[0.011926690675758975, 0.032585771408010535, -0.08467378092322836,
        0.0, -0.001352055042362226],
       [0.07886219957032432, 0.013794891785999885, -0.1693475618464562,
        0.0, -0.009464385296535574],
       [-0.24559246718042357, 0.2506941955920627, 0.0, 0.0,
        0.016224660508346693],
       ...,
       [-0.08251322732820968, 0.022071826857599813,
        -0.056449187282152154, 0.0, -0.041913706313228966],
       [-0.2631174004182734, 0.7561837708206745, 0.11289837456430404,
        0.0, -0.04732192648267786],
       [0.2592229708098624, 0.13660671217271234, 0.14112296820538026,
        0.0, -0.006760275211811123]], dtype=object)

In [5]:
from sklearn.ensemble import RandomForestClassifier

# אימון מודל עץ החלטה על 5 הפיצ'רים החשובים בלבד
model_top5 = RandomForestClassifier()
X_train_top5, y_train = prepare_weighted_training_data(train_pairs, top_5_features, top_5_feature_importance)
model_top5.fit(X_train_top5, y_train)



RandomForestClassifier()

In [6]:
from sklearn.preprocessing import StandardScaler

# יצירת זוגות אקראיים עבור קבוצת הבדיקה
def create_test_pairs(data, numeric_columns, num_pairs=100):
    pairs = []
    pages = data['page_num'].unique()

    scaler = StandardScaler()
    data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
    
    for page in pages:
        page_data = data[data['page_num'] == page]
        indices = page_data.index.values

        if len(indices) < 2:
            continue

        for _ in range(num_pairs):
            idx_a, idx_b = np.random.choice(indices, size=2, replace=False)
            hotel_a_features = page_data.loc[idx_a, numeric_columns].values
            hotel_b_features = page_data.loc[idx_b, numeric_columns].values
            
            if hotel_a_features.shape != hotel_b_features.shape:
                continue

            pairs.append({
                'page_num': page,
                'hotel_a_features': hotel_a_features,
                'hotel_b_features': hotel_b_features,
                'idx_a': idx_a,
                'idx_b': idx_b
            })

    return pd.DataFrame(pairs)

# חיזוי על זוגות הבדיקה
def predict_test_pairs(pairs, important_features, feature_importance, model_weighted, all_features):
    predictions = []

    for _, row in pairs.iterrows():
        hotel_a_features = pd.Series(row['hotel_a_features'], index=all_features)
        hotel_b_features = pd.Series(row['hotel_b_features'], index=all_features)
        
        hotel_a_features = hotel_a_features[important_features].values
        hotel_b_features = hotel_b_features[important_features].values

        diff_features = (hotel_a_features - hotel_b_features) * feature_importance

        # Ensure diff_features has the correct shape for the model
        diff_features = np.array(diff_features).reshape(1, -1)

        prediction = model_weighted.predict(diff_features)[0]
        predictions.append(prediction)

    pairs['prediction'] = predictions
    return pairs

# יצירת זוגות אקראיים עבור קבוצת הבדיקה
test_pairs = create_test_pairs(test_data, numeric_columns)
test_pairs = predict_test_pairs(test_pairs, top_5_features, top_5_feature_importance, model_top5, numeric_columns)

# מיון יציב של בתי המלון לפי התחזיות
def stable_sort_hotels(page_data, predictions):
    hotels = list(page_data['idx_a'].unique()) + list(page_data['idx_b'].unique())
    hotels = list(set(hotels))
    scores = {hotel: 0 for hotel in hotels}

    for _, row in predictions.iterrows():
        hotel_a = row['idx_a']
        hotel_b = row['idx_b']
        prediction = row['prediction']

        if prediction == 1:
            scores[hotel_a] += 1
        else:
            scores[hotel_b] += 1

    sorted_hotels = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    return [hotel for hotel, score in sorted_hotels]

# מיון בתי המלון בקבוצת הבדיקה
sorted_hotels = {}
for page in test_pairs['page_num'].unique():
    page_data = test_pairs[test_pairs['page_num'] == page]
    sorted_hotels[page] = stable_sort_hotels(page_data, page_data)

# הצגת התוצאות עבור עמוד אחד לדוגמה
sorted_hotels[list(sorted_hotels.keys())[0]]

# שמירת הנתונים לקובץ CSV
page_data.to_csv('booking_sorted_hotels.csv', index=False)


C:\Users\ygastfraind\AppData\Local\Temp\ipykernel_10384\3813585845.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_columns] = scaler.fit_transform(data[numeric_columns])


In [7]:
import numpy as np

def calculate_error(original_order, predicted_order):
    original_index = {hotel: idx for idx, hotel in enumerate(original_order)}
    predicted_index = {hotel: idx for idx, hotel in enumerate(predicted_order)}
    
    error = 0
    for hotel in original_order:
        if hotel in predicted_index:
            error += (original_index[hotel] - predicted_index[hotel])**2
    return error

errors = []

for page in test_data['page_num'].unique():
    # ביצוע שאפל אקראי של הנתונים בעמוד
    page_data = test_data[test_data['page_num'] == page].sample(frac=1).reset_index(drop=True)
    
    # יצירת זוגות אקראיים מחדש עם נרמול
    test_pairs_shuffled = create_test_pairs(page_data, numeric_columns)
    
    # ביצוע תחזיות על זוגות הבדיקה המשופעים
    test_pairs_shuffled = predict_test_pairs(test_pairs_shuffled, top_5_features, top_5_feature_importance, model_top5, numeric_columns)
    
    # מיון מחדש של בתי המלון בעמוד
    sorted_hotels_shuffled = stable_sort_hotels(test_pairs_shuffled, test_pairs_shuffled)
    sorted_hotels_shuffled
    # חישוב השגיאה עבור העמוד המשופע
    original_order = page_data.index.tolist()
    page_error = calculate_error(original_order, sorted_hotels_shuffled)
    errors.append(page_error)
    print(f'Error for page {page}: {page_error}')

mean_error = np.mean(errors)
std_error = np.std(errors)
print(sorted_hotels_shuffled)
print(f'Mean Error: {mean_error}')
print(f'Standard Deviation of Error: {std_error}')


Error for page 1: 331765
Error for page 2: 365750
Error for page 3: 301325
Error for page 4: 278152
Error for page 5: 198355
Error for page 11: 286630
Error for page 12: 370604
Error for page 13: 276204
Error for page 14: 217078
Error for page 15: 181465
Error for page 21: 327193
Error for page 22: 343274
Error for page 23: 304839
Error for page 24: 198297
Error for page 25: 247982
Error for page 31: 374243
Error for page 32: 373007
Error for page 33: 270311
Error for page 34: 307524
Error for page 35: 245830
Error for page 41: 382447
Error for page 42: 370842
Error for page 43: 261013
Error for page 44: 269012
Error for page 45: 175831
Error for page 51: 326785
Error for page 52: 333092
Error for page 53: 222948
Error for page 54: 260515
Error for page 55: 130690
Error for page 61: 424492
Error for page 62: 389749
Error for page 63: 292448
Error for page 64: 359724
Error for page 65: 282069
Error for page 71: 311474
Error for page 72: 384727
Error for page 73: 354664
Error for page 74